In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sumathisanthosh/gestational-diabetes-mellitus-gdm-data-set")

print("Path to dataset files:", path)

100%|██████████| 194k/194k [00:00<00:00, 33.7MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/sumathisanthosh/gestational-diabetes-mellitus-gdm-data-set/versions/1


In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [4]:
data = pd.read_excel(path + "/Gestational Diabetic Dat Set.xlsx")

In [5]:
data.head()

,Case Number,Age,No of Pregnancy,Gestation in previous Pregnancy,BMI,HDL,Family History,unexplained prenetal loss,Large Child or Birth Default,PCOS,Sys BP,Dia BP,OGTT,Hemoglobin,Sedentary Lifestyle,Prediabetes,Class Label(GDM /Non GDM)
0,1,22,2,1,NaN,55.0,0,0,0,0,102.0,69,NaN,12.0,0,0,0
1,2,26,2,1,NaN,53.0,0,0,0,0,101.0,63,NaN,12.4,0,0,0
2,3,29,1,0,NaN,50.0,0,0,0,0,118.0,79,NaN,14.3,0,0,0
3,4,28,2,1,NaN,51.0,0,0,0,0,99.0,70,NaN,15.0,0,0,0
4,5,21,2,1,NaN,52.0,0,0,0,0,116.0,65,NaN,15.0,0,0,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3525 entries, 0 to 3524
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Case Number                      3525 non-null   int64  
 1   Age                              3525 non-null   int64  
 2   No of Pregnancy                  3525 non-null   int64  
 3   Gestation in previous Pregnancy  3525 non-null   int64  
 4   BMI                              2444 non-null   float64
 5   HDL                              2524 non-null   float64
 6   Family History                   3525 non-null   int64  
 7   unexplained prenetal loss        3525 non-null   int64  
 8   Large Child or Birth Default     3525 non-null   int64  
 9   PCOS                             3525 non-null   int64  
 10  Sys BP                           1820 non-null   float64
 11  Dia BP                           3525 non-null   int64  
 12  OGTT                

In [7]:
data.isnull().sum()

,0
Case Number,0
Age,0
No of Pregnancy,0
Gestation in previous Pregnancy,0
BMI,1081
HDL,1001
Family History,0
unexplained prenetal loss,0
Large Child or Birth Default,0
PCOS,0


In [8]:
data.columns

Index(['Case Number', 'Age', 'No of Pregnancy',
       'Gestation in previous Pregnancy', 'BMI', 'HDL', 'Family History',
       'unexplained prenetal loss', 'Large Child or Birth Default', 'PCOS',
       'Sys BP', 'Dia BP', 'OGTT', 'Hemoglobin', 'Sedentary Lifestyle',
       'Prediabetes', 'Class Label(GDM /Non GDM)'],
      dtype='object')

In [9]:
missing_data = data.isnull().mean() * 100
print(missing_data)

Case Number                         0.000000
Age                                 0.000000
No of Pregnancy                     0.000000
Gestation in previous Pregnancy     0.000000
BMI                                30.666667
HDL                                28.397163
Family History                      0.000000
unexplained prenetal loss           0.000000
Large Child or Birth Default        0.000000
PCOS                                0.000000
Sys BP                             48.368794
Dia BP                              0.000000
OGTT                               14.553191
Hemoglobin                          0.000000
Sedentary Lifestyle                 0.000000
Prediabetes                         0.000000
Class Label(GDM /Non GDM)           0.000000
dtype: float64


In [11]:
data.drop(['Sys BP'], axis=1, inplace=True)

In [12]:
imputer = SimpleImputer(strategy='mean')
data[['BMI', 'HDL', 'OGTT']] = imputer.fit_transform(data[['BMI', 'HDL', 'OGTT']])

In [13]:
print("\nRemaining Missing Data:\n", data.isnull().sum())


Remaining Missing Data:
 Case Number                        0
Age                                0
No of Pregnancy                    0
Gestation in previous Pregnancy    0
BMI                                0
HDL                                0
Family History                     0
unexplained prenetal loss          0
Large Child or Birth Default       0
PCOS                               0
Dia BP                             0
OGTT                               0
Hemoglobin                         0
Sedentary Lifestyle                0
Prediabetes                        0
Class Label(GDM /Non GDM)          0
dtype: int64


In [15]:
X = data.drop(['Class Label(GDM /Non GDM)'], axis=1)
y = data['Class Label(GDM /Non GDM)']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [19]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

Epoch 1/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.7564 - loss: 0.5268 - val_accuracy: 0.9453 - val_loss: 0.1848
Epoch 2/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9590 - loss: 0.1661 - val_accuracy: 0.9534 - val_loss: 0.1445
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9640 - loss: 0.1092 - val_accuracy: 0.9514 - val_loss: 0.1337
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9759 - loss: 0.0863 - val_accuracy: 0.9575 - val_loss: 0.1238
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9735 - loss: 0.0804 - val_accuracy: 0.9575 - val_loss: 0.1180
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9758 - loss: 0.0720 - val_accuracy: 0.9595 - val_loss: 0.1074
Epoch 7/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9782 - loss: 0.0599 - val_accuracy: 0.9595 - val_loss: 0.1093
Epoch 8/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9813 - loss: 0.0633 - val_accuracy: 0.9656 - val_lo

In [23]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [24]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nROC-AUC Score:", roc_auc_score(y_test, model.predict(X_test)))

Confusion Matrix:
 [[643  20]
 [ 12 383]]

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.98       663
           1       0.95      0.97      0.96       395

    accuracy                           0.97      1058
   macro avg       0.97      0.97      0.97      1058
weighted avg       0.97      0.97      0.97      1058

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

ROC-AUC Score: 0.9959104186952288
